# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

In [2]:
import numpy as np

## imports for Python, Pandas

In [3]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [4]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [5]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,shortname,state
0,Dade,12345,Rick Scott,FL,Florida
1,Broward,40000,Rick Scott,FL,Florida
2,Palm Beach,60000,Rick Scott,FL,Florida
3,Summit,1234,John Kasich,OH,Ohio
4,Cuyahoga,1337,John Kasich,OH,Ohio


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [7]:
# load json as string
aes = json.load((open('data/world_bank_projects.json')))

In [8]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [10]:
# load as Pandas dataframe
df = pd.read_json('data/world_bank_projects.json')

In [11]:
df.countryname.value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

In [12]:
finalval =json_normalize(aes, 'mjtheme_namecode')
finalval.code.value_counts().head(10)
#finalval.groupby(['name']).size().sort_values()

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64

Series([], dtype: int64)

In [64]:
finalval.groupby(['code','name']).size()

code  name                                        
1                                                       5
      Economic management                              33
10                                                     14
      Rural development                               202
11                                                     27
      Environment and natural resources management    223
2                                                      15
      Public sector governance                        184
3                                                       3
      Rule of law                                      12
4                                                      16
      Financial and private sector development        130
5                                                       5
      Trade and integration                            72
6                                                      10
      Social protection and risk management           158
7                    

In [66]:
a=finalval.copy()
a=a.replace(to_replace='',value=np.NaN).sort_values(by='code').ffill()
a=a.groupby(['code','name']).size()
a

code  name                                        
1     Economic management                              38
10    Rural development                               216
11    Environment and natural resources management    250
2     Public sector governance                        199
3     Rule of law                                      15
4     Financial and private sector development        146
5     Trade and integration                            77
6     Social protection and risk management           168
7     Social dev/gender/inclusion                     130
8     Human development                               210
9     Urban development                                50
dtype: int64

In [18]:
df['_id']

0    {'$oid': '52b213b38594d8a2be17c780'}
1    {'$oid': '52b213b38594d8a2be17c781'}
Name: _id, dtype: object